In [ ]:
#This script uses the function in the "Block_structure_periodic" module and "Solvers_residual" module to calculate the
#electric field in a silicon microstrip detector using a 2-mesh GMG-method

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Block_structure_periodic import *
from Solvers_residual import *
from CG_solver import *
from scipy import interpolate

KeyboardInterrupt: 

In [ ]:
#Define the sizes of the different parts of the domain

pitch = 100
height = 600
mesh_size_1 = 0.5
mesh_size_2 = 1

N_x_1 = int(pitch/mesh_size_1)
N_y_1 = int(height/mesh_size_1)
active_cells_1 = int(N_x_1*N_y_1)
N_x_2 = int(pitch/mesh_size_2)
N_y_2 = int(height/mesh_size_2)
active_cells_2 = int(N_x_2*N_y_2)

#define the reference coordinates for the fine and the coarse mesh
x_C = np.arange(0.5, 100.5, 1)
y_C = np.arange(0.5, 600.5, 1)
x_F = np.arange(0.25, 100.25, 0.5)
y_F = np.arange(0.25, 600.25, 0.5)

In [ ]:
#Define the matrix of coefficients and the matrix of positions for the different blocks
#The location of the electrode is specified in the relevant function in the module "Block_structure_periodic"
#The potential is set to 0 at the p+ implant

#matrices for the fine mesh
coeff_central, position_central = von_neumann_bottom_block_periodic_inactivated(pitch, height, mesh_size_1)
coeff_central = np.array(coeff_central)
position_central = np.array(position_central)

#matrices on the coarse mesh
coeff_central_C, position_central_C = von_neumann_bottom_block_periodic_Coarse_inactivated(pitch, height, mesh_size_2)
coeff_central_C = np.array(coeff_central_C)
position_central_C = np.array(position_central_C)

In [ ]:
#load guess

potential = np.loadtxt("potential_von_neumann_partial_depletion_guess.csv", delimiter = ",")

#create correction vector
correction = np.zeros(active_cells_2)

In [ ]:
#define the source terms

source_central = np.zeros(int(N_x_1*N_y_1))

#define the charge distribution
#donor concentration/relaive_permitivity of silicon
#they will contribute as positive charges
N_D = 3.5/11.7
#Ignore negative charge in p+ implant
N_A = 0

#bulk concentration
for i in range(0, int((1/3)*N_y_1*N_x_1 - 2*N_x_1 - 1)):
    source_central[i] = -N_D 

#define the implant
#thickness of the implant is 1 micron
#width of the implant is 20 microns
#define the limits of the for loops accordingly
for i in range(int((1/3)*N_y_1*N_x_1 - 2*N_x_1), int((1/3)*N_y_1*N_x_1 - 2*N_x_1 + (2/5)*N_x_1)):
    source_central[i] = -N_D
    
for i in range(int((1/3)*N_y_1*N_x_1 - 2*N_x_1 + (2/5)*N_x_1), int((1/3)*N_y_1*N_x_1 - 2*N_x_1 + (3/5)*N_x_1)):
    source_central[i] = -N_A
    
for i in range(int((1/3)*N_y_1*N_x_1 - 2*N_x_1 + (3/5)*N_x_1), int((1/3)*N_y_1*N_x_1 - N_x_1)):
    source_central[i] = -N_D
    
for i in range(int((1/3)*N_y_1*N_x_1 - N_x_1), int((1/3)*N_y_1*N_x_1 - N_x_1 + (2/5)*N_x_1)):
    source_central[i] = -N_D
    
for i in range(int((1/3)*N_y_1*N_x_1 - N_x_1 + (2/5)*N_x_1), int((1/3)*N_y_1*N_x_1 - N_x_1 + (3/5)*N_x_1)):
    source_central[i] = -N_A
    
for i in range(int((1/3)*N_y_1*N_x_1 - N_x_1 + (3/5)*N_x_1), int((1/3)*N_y_1*N_x_1)):
    source_central[i] = -N_D

#These are the transition cells of the domain, they are nedded in the function that updates the source terms
#This part of the code is a relict of a division in blocks of the solution domain before the GMG-method was applied
B_bottom = np.zeros(N_x_1)
B_bottom[0:N_x_1] = N_D

B_top = np.zeros(N_x_1)

In [ ]:
#SOR solution in GMG set-up

#residual requirement
tolerance = 1

#counter variables for the iterations over fine and coarse mesh
iteration_F = 1
iteration_C = 1
corr = 0 #variable to control the re-initialisation of the correction vector

#residual on the fine mesh
R_F = 100

#update the source
source_central = update_bottom_top_source(source_central, pitch, height, mesh_size_1, B_bottom, B_top)

while R_F > tolerance:
    
    #apply one SOR iteartion over the fine mesh
    potential = SOR_one_iter(coeff_central, position_central, source_central, potential, 1.8, active_cells_1)
    
    #calculate the global residual over the fine mesh
    R_F = global_residual(coeff_central, position_central, source_central, potential, active_cells_1)
    
    print("Iteration F = " + str(iteration_F))
    print("R_F = " + str(R_F))
    print(potential[0])
    
    #iteration count
    iteration_F = iteration_F + 1
    
    #iteration over the coarse mesh
    if int(iteration_F%30) == 0:
        
        #periodically re-initialise the correction vector
        corr = corr + 1
        if corr % 2 == 0:
            correction = np.zeros(active_cells_2)
        
        #calculate the global vector residual on the fine mesh
        R_vect = global_vector_residual(coeff_central, position_central, source_central, potential, active_cells_1, 0)
        
        #distribute the global vector residual over the fine mesh domain
        R_vect_flat = np.zeros((N_y_1, N_x_1))
        #flatten the R_vect
        for i in range(N_y_1):
            for j in range(N_x_1):
                location = i*N_x_1 + j
                R_vect_flat[i, j] = R_vect[location]
                
        #create the residual vector in the coarse mesh interpolating the residuals on the fine mesh to the coarse mesh
        f = interpolate.RectBivariateSpline(y_F, x_F, R_vect_flat)
        #this is an object that performs a cubic spline over the fine mesh domain

        #used here to create the residual vector on the coarse mesh
        R_C_vect = []
        for i in range(N_y_2):
            for j in range(N_x_2):
                x_new = x_C[j]
                y_new = y_C[i]

                R_new = f(y_new, x_new)

                R_C_vect.append(R_new)
        
        #perform a number of iterations on the coarse mesh
        for i in range(60):
            
            #SOR iteration
            correction = SOR_one_iter(coeff_central_C, position_central_C, R_C_vect, correction, 1.8, active_cells_2)
            #ensure that the correction is zero at the strip electrode and the p+ implant
            correction[int(((1/3)*N_y_1 - 1)*N_x_1 + (2/5)*N_x_1):int(((1/3)*N_y_1 - 1)*N_x_1 + (3/5)*N_x_1)] = 0
            correction[int(((1/3)*N_y_1)*N_x_1 + (2/5)*N_x_1):int(((1/3)*N_y_1)*N_x_1 + (3/5)*N_x_1)] = 0
            
            #calculate the residual
            R_C = global_residual(coeff_central_C, position_central_C, R_C_vect, correction, active_cells_2)
           
            print("Iteration C = " + str(iteration_C))
            print("R_C = " + str(R_C))
            print(correction[0])
            
            iteration_C = iteration_C + 1
            
        #distribute the correction vector over the domain of the coarse mesh
        correction_flat = np.zeros((N_y_2, N_x_2))
        for i in range(N_y_2):
            for j in range(N_x_2):
                location = i*N_x_2 + j
                correction_flat[i, j] = correction[location]
        
        #create cubic spline interpolator object over the coarse mesh
        g = interpolate.RectBivariateSpline(y_C, x_C, correction_flat)
        
        #interpolate the correction vector on the fine mesh
        correction_F = []
        for i in range(N_y_1):
            for j in range(N_x_1):
                x_new = x_F[j]
                y_new = y_F[i]

                correction_new = g(y_new, x_new)

                correction_F.append(correction_new)
        
        #add the correction to the potential on the fine mesh
        for n in range(active_cells_2):
            potential[n] = potential[n] + correction_F[n]

In [ ]:
#plot the potential

potential_plane = np.zeros((N_y_1, N_x_1))

for i in range(0, N_y_1):
    for j in range(0, N_x_1):
        
        location = i*N_x_1 + j
        
        potential_plane[i, j] = potential[location]
 

x = np.arange(0, 100, 0.5)
y = np.arange(0, 600, 0.5)

X, Y = np.meshgrid(x, y)

fig, ax = plt.subplots(1, 1, figsize = (8, 8))
cax = ax.contour(X, Y, potential_plane, 40)
fig.colorbar(cax)

In [ ]:
#save the potential vector
np.savetxt("potential_R_F=1.csv", potential, delimiter = ",")